# Wasserstein Generative Adversarial Network - WGAN
---
The Wasserstein GAN, or WGAN for short, was introduced by Martin Arjovsky, et al. in their 2017 paper titled [Wasserstein GAN](https://arxiv.org/abs/1701.07875).

The Wasserstein GAN (WGAN) extends the traditional GAN by introducing an alternative approach to training the generator. Its goal is to improve the generator's ability to approximate the data distribution observed in the training dataset.

Rather than using a discriminator to classify generated images as real or fake, the WGAN replaces it with a critic. The critic evaluates images by scoring their "realness" or "fakeness," providing a continuous metric instead of binary classification.

This modification is based on a theoretical principle: training the generator should minimize the distance between the data distribution in the training set and the distribution of generated samples.

WGAN offers several advantages. Its training process is more stable, less sensitive to model architecture, and less dependent on hyperparameter configurations. Most importantly, the critic's loss correlates with the quality of images produced by the generator, making it a more reliable indicator of training progress.

## Wasserstein GAN Implementation Details
---
Although the theoretical grounding for the WGAN is dense, the implementation of a WGAN requires a few minor changes to the standard Deep Convolutional GAN, or DCGAN.

The image below provides a summary of the main training loop for training a WGAN, taken from the paper. Note the listing of recommended hyperparameters used in the model.


![gan-algorithm](plots/wgan-algorithm.png)

The differences in implementation for the WGAN are as follows:

1. Use a linear activation function in the output layer of the critic model (instead of sigmoid).
2. Use -1 labels for real images and 1 labels for fake images (instead of 1 and 0).
3. Use Wasserstein loss to train the critic and generator models.
4. Constrain critic model weights to a limited range after each mini batch update (e.g. [-0.01,0.01]).
5. Update the critic model more times than the generator each iteration (e.g. 5).
6. Use the RMSProp version of gradient descent with a small learning rate and no momentum (e.g. 0.00005).

**NOTE** This code will to some extend be a modified version of that implemented in the [DCGAN](https://github.com/sulaiman-shamasna/GANs/blob/main/DCGANs.ipynb) notebook.


In [76]:
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint

In [77]:
from matplotlib import pyplot

In [78]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [79]:
import tensorflow as tf

# Define the Wasserstein loss function
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

## How to Train a Wasserstein GAN Model
---
Now that we know the specific implementation details for the WGAN, we can implement the model for image generation.

In this section, we will develop a WGAN to generate a single handwritten digit (‘7’) from the [MNIST dataset](https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-fashion-mnist-clothing-classification/). This is a good test problem for the WGAN as it is a small dataset requiring a modest mode that is quick to train.

The first step is to define the models.

The critic model takes as input one 28×28 grayscale image and outputs a score for the realness or fakeness of the image. It is implemented as a modest convolutional neural network using best practices for DCGAN design such as using the [LeakyReLU activation](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/) function with a slope of 0.2, [batch normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/), and using a [2×2 stride to downsample](https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/).

The critic model makes use of the new ClipConstraint weight constraint to clip model weights after mini-batch updates and is optimized using the custom wasserstein_loss() function, the RMSProp version of stochastic gradient descent with a learning rate of 0.00005.

The ```define_critic()``` function below implements this, defining and compiling the critic model and returning it. The input shape of the image is parameterized as a default function argument to make it clear.

In [80]:
# define the standalone critic model
def define_critic(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# weight constraint
	const = ClipConstraint(0.01)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const, input_shape=in_shape))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, kernel_constraint=const))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# scoring, linear activation
	model.add(Flatten())
	model.add(Dense(1))
	# compile model
	opt = RMSprop(learning_rate=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

In [81]:
define_critic().summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)              │ (None, 14, 14, 64)     │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_33 (LeakyReLU)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 7, 7, 64)       │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_34 (LeakyReLU)      │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │         3,137 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,337 (274.75 KB)

 Trainable params: 70,081 (273.75 KB)

 Non-trainable params: 256 (1.00 KB)

The generator model takes as input a point in the latent space and outputs a single 28×28 grayscale image.

This is achieved by using a fully connected layer to interpret the point in the latent space and provide sufficient activations that can be reshaped into many copies (in this case, 128) of a low-resolution version of the output image (e.g. 7×7). This is then upsampled two times, doubling the size and quadrupling the area of the activations each time using transpose convolutional layers.

The model uses best practices such as the LeakyReLU activation, a kernel size that is a factor of the stride size, and a hyperbolic tangent (tanh) activation function in the output layer.

The define_generator() function below defines the generator model but intentionally does not compile it as it is not trained directly, then returns the model. The size of the latent space is parameterized as a function argument.

In [82]:
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

Next, a GAN model can be defined that combines both the generator model and the critic model into one larger model.

This larger model will be used to train the model weights in the generator, using the output and error calculated by the critic model. The critic model is trained separately, and as such, the model weights are marked as not trainable in this larger GAN model to ensure that only the weights of the generator model are updated. This change to the trainability of the critic weights only has an effect when training the combined GAN model, not when training the critic standalone.

This larger GAN model takes as input a point in the latent space, uses the generator model to generate an image, which is fed as input to the critic model, then output scored as real or fake. The model is fit using RMSProp with the custom wasserstein_loss() function.

The define_gan() function below implements this, taking the already defined generator and critic models as input.

In [83]:
# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
	# make weights in the critic not trainable
	for layer in critic.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(learning_rate=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model

Now that we have defined the GAN model, we need to train it. But, before we can train the model, we require input data.

The first step is to load and [scale the MNIST dataset](https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/). The whole dataset is loaded via a call to the load_data() Keras function, then a subset of the images is selected (about 5,000) that belongs to class 7, e.g. are a handwritten depiction of the number seven. Then the pixel values must be scaled to the range [-1,1] to match the output of the generator model.

The ```load_real_samples()``` function below implements this, returning the loaded and scaled subset of the MNIST training dataset ready for modeling.

In [84]:
# load images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# select all of the examples for a given class
	selected_ix = trainy == 7
	X = trainX[selected_ix]
	# expand to 3d, e.g. add channels
	X = expand_dims(X, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

We will require one batch (or a half) batch of real images from the dataset each update to the GAN model. A simple way to achieve this is to select a [random sample](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/) of images from the dataset each time.

The ```generate_real_samples()``` function below implements this, taking the prepared dataset as an argument, selecting and returning a random sample of images and their corresponding label for the critic, specifically target=-1 indicating that they are real images.

In [85]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels, -1 for 'real'
	y = -ones((n_samples, 1))
	return X, y
 

Next, we need inputs for the generator model. These are random points from the latent space, specifically [Gaussian distributed random variables](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/).

The generate_latent_points() function implements this, taking the size of the latent space as an argument and the number of points required, and returning them as a batch of input samples for the generator model.

In [86]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

Next, we need to use the points in the latent space as input to the generator in order to generate new images.

The ```generate_fake_samples()``` function below implements this, taking the generator model and size of the latent space as arguments, then generating points in the latent space and using them as input to the generator model.

The function returns the generated images and their corresponding label for the critic model, specifically target=1 to indicate they are fake or generated.

In [87]:
# Use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # directly call the generator
    X = generator(x_input, training=False)
    # create class labels with 1.0 for 'fake'
    y = tf.ones((n_samples, 1))
    return X, y

We need to record the performance of the model. Perhaps the most reliable way to evaluate the performance of a GAN is to use the generator to generate images, and then review and subjectively evaluate them.

The ```summarize_performance()``` function below takes the generator model at a given point during training and uses it to generate 100 images in a 10×10 grid, that are then plotted and saved to file. The model is also saved to file at this time, in case we would like to use it later to generate more images.

In [88]:
import os
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	filename1_dir = os.path.join('wgan_results', filename1)	
	pyplot.savefig(filename1_dir)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	filename2_dir = os.path.join('wgan_results', filename2)
	g_model.save(filename2_dir)
	print('>Saved: %s and %s' % (filename1, filename2_dir))

In addition to image quality, it is a good idea to keep track of the loss and accuracy of the model over time.

The loss for the critic for real and fake samples can be tracked for each model update, as can the loss for the generator for each update. These can then be used to create line plots of loss at the end of the training run. The plot_history() function below implements this and saves the results to file.

In [89]:
import os
result_path = os.path.join('wgan_results', 'plot_line_plot_loss.png')
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	# plot history
	pyplot.plot(d1_hist, label='crit_real')
	pyplot.plot(d2_hist, label='crit_fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	pyplot.savefig(result_path)
	pyplot.close()

We are now ready to fit the GAN model.

The model is fit for 10 training epochs, which is arbitrary, as the model begins generating plausible number-7 digits after perhaps the first few epochs. A batch size of 64 samples is used, and each training epoch involves 6,265/64, or about 97, batches of real and fake samples and updates to the model. The model is therefore trained for 10 epochs of 97 batches, or 970 iterations.

First, the critic model is updated for a half batch of real samples, then a half batch of fake samples, together forming one batch of weight updates. This is then repeated n_critic (5) times as required by the WGAN algorithm.

The generator is then updated via the composite GAN model. Importantly, the target label is set to -1 or real for the generated samples. This has the effect of updating the generator toward getting better at generating real samples on the next batch.

The train() function below implements this, taking the defined models, dataset, and size of the latent dimension as arguments and parameterizing the number of epochs and batch size with default arguments. The generator model is saved at the end of training.

The performance of the critic and generator models is reported each iteration. Sample images are generated and saved every epoch, and line plots of model performance are created and saved at the end of the run.

In [90]:
# Define the optimizers outside of the train_step function
critic_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
generator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00005)

@tf.function
def train_step(g_model, c_model, gan_model, real_images, latent_dim, n_batch, n_critic, critic_optimizer, generator_optimizer):
    c1_losses, c2_losses = [], []

    # Train the critic more frequently than the generator
    for _ in range(n_critic):
        # Generate fake images
        fake_images, y_fake = generate_fake_samples(g_model, latent_dim, n_batch // 2)
        y_real = -tf.ones((n_batch // 2, 1))  # Label for real images in WGAN

        # Update critic on real images
        with tf.GradientTape() as c_tape:
            real_output = c_model(real_images, training=True)
            fake_output = c_model(fake_images, training=True)
            c_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

        # Apply gradients to critic
        c_gradients = c_tape.gradient(c_loss, c_model.trainable_variables)
        critic_optimizer.apply_gradients(zip(c_gradients, c_model.trainable_variables))
        c1_losses.append(c_loss)

    # Prepare latent points and inverted labels for generator update
    latent_points = generate_latent_points(latent_dim, n_batch)
    y_gan = -tf.ones((n_batch, 1))

    # Update generator via critic’s loss
    with tf.GradientTape() as g_tape:
        generated_images = g_model(latent_points, training=True)
        fake_output = c_model(generated_images, training=False)
        g_loss = -tf.reduce_mean(fake_output)

    # Apply gradients to generator
    g_gradients = g_tape.gradient(g_loss, g_model.trainable_variables)
    generator_optimizer.apply_gradients(zip(g_gradients, g_model.trainable_variables))
    c2_losses.append(g_loss)

    return tf.reduce_mean(c1_losses), tf.reduce_mean(c2_losses), g_loss


In [91]:
# Train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    
    # Lists for keeping track of loss
    c1_hist, c2_hist, g_hist = list(), list(), list()
    
    for i in range(n_steps):
        # Get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, n_batch // 2)
        
        # Perform a training step
        c1_loss, c2_loss, g_loss = train_step(g_model, c_model, gan_model, X_real, latent_dim, n_batch, n_critic, critic_optimizer, generator_optimizer)
        
        # Track losses
        c1_hist.append(c1_loss)
        c2_hist.append(c2_loss)
        g_hist.append(g_loss)
        
        print(f'>{i+1}, c1={c1_loss:.3f}, c2={c2_loss:.3f}, g={g_loss:.3f}')
        
        # Summarize performance
        if (i+1) % bat_per_epo == 0:
            summarize_performance(i, g_model, latent_dim)
    
    plot_history(c1_hist, c2_hist, g_hist)


Now that all of the functions have been defined, we can create the models, load the dataset, and begin the training process.

In [92]:
# size of the latent space
latent_dim = 50
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, critic, gan_model, dataset, latent_dim)

(6265, 28, 28, 1)
>1, c1=0.132, c2=-0.020, g=-0.020
>2, c1=0.273, c2=-0.065, g=-0.065
>3, c1=0.428, c2=-0.108, g=-0.108
>4, c1=0.431, c2=-0.149, g=-0.149
>5, c1=0.529, c2=-0.193, g=-0.193
>6, c1=0.539, c2=-0.239, g=-0.239
>7, c1=0.420, c2=-0.290, g=-0.290
>8, c1=1.075, c2=-0.348, g=-0.348
>9, c1=0.826, c2=-0.409, g=-0.409
>10, c1=0.897, c2=-0.473, g=-0.473
>11, c1=0.943, c2=-0.542, g=-0.542
>12, c1=1.009, c2=-0.630, g=-0.630
>13, c1=1.003, c2=-0.719, g=-0.719
>14, c1=1.291, c2=-0.816, g=-0.816
>15, c1=1.356, c2=-0.914, g=-0.914
>16, c1=1.201, c2=-1.043, g=-1.043
>17, c1=1.414, c2=-1.182, g=-1.182
>18, c1=1.386, c2=-1.333, g=-1.333
>19, c1=1.382, c2=-1.501, g=-1.501
>20, c1=1.527, c2=-1.672, g=-1.672
>21, c1=1.478, c2=-1.872, g=-1.872
>22, c1=1.658, c2=-2.079, g=-2.079
>23, c1=1.610, c2=-2.319, g=-2.319
>24, c1=1.729, c2=-2.567, g=-2.567
>25, c1=1.612, c2=-2.824, g=-2.824
>26, c1=1.888, c2=-3.094, g=-3.094
>27, c1=1.983, c2=-3.430, g=-3.430
>28, c1=1.809, c2=-3.791, g=-3.791
>29, c1=2.1

>Saved: generated_plot_0097.png and wgan_results\model_0097.h5
>98, c1=4.841, c2=-53.072, g=-53.072
>99, c1=4.665, c2=-53.298, g=-53.298
>100, c1=4.788, c2=-53.570, g=-53.570
>101, c1=4.514, c2=-53.818, g=-53.818
>102, c1=4.604, c2=-53.976, g=-53.976
>103, c1=4.631, c2=-53.996, g=-53.996
>104, c1=5.161, c2=-54.203, g=-54.203
>105, c1=5.033, c2=-54.352, g=-54.352
>106, c1=4.962, c2=-54.528, g=-54.528
>107, c1=5.104, c2=-54.678, g=-54.678
>108, c1=5.000, c2=-54.710, g=-54.710
>109, c1=4.786, c2=-54.800, g=-54.800
>110, c1=5.147, c2=-54.906, g=-54.906
>111, c1=4.995, c2=-54.998, g=-54.998
>112, c1=5.325, c2=-55.214, g=-55.214
>113, c1=5.302, c2=-55.383, g=-55.383
>114, c1=5.159, c2=-55.529, g=-55.529
>115, c1=5.327, c2=-55.750, g=-55.750
>116, c1=5.307, c2=-55.889, g=-55.889
>117, c1=5.250, c2=-55.983, g=-55.983
>118, c1=5.500, c2=-55.889, g=-55.889
>119, c1=5.385, c2=-56.135, g=-56.135
>120, c1=5.046, c2=-56.214, g=-56.214
>121, c1=5.338, c2=-56.287, g=-56.287
>122, c1=5.270, c2=-56.359,

>Saved: generated_plot_0194.png and wgan_results\model_0194.h5
>195, c1=7.428, c2=-56.485, g=-56.485
>196, c1=7.311, c2=-56.452, g=-56.452
>197, c1=7.358, c2=-56.412, g=-56.412
>198, c1=7.362, c2=-56.244, g=-56.244
>199, c1=7.608, c2=-56.211, g=-56.211
>200, c1=7.395, c2=-56.082, g=-56.082
>201, c1=7.337, c2=-56.052, g=-56.052
>202, c1=7.842, c2=-55.988, g=-55.988
>203, c1=7.442, c2=-55.987, g=-55.987
>204, c1=7.586, c2=-55.849, g=-55.849
>205, c1=7.537, c2=-55.731, g=-55.731
>206, c1=7.618, c2=-55.699, g=-55.699
>207, c1=7.645, c2=-55.657, g=-55.657
>208, c1=7.261, c2=-55.690, g=-55.690
>209, c1=7.737, c2=-55.840, g=-55.840
>210, c1=7.745, c2=-55.799, g=-55.799
>211, c1=7.926, c2=-55.748, g=-55.748
>212, c1=7.787, c2=-55.515, g=-55.515
>213, c1=7.570, c2=-55.356, g=-55.356
>214, c1=7.621, c2=-55.285, g=-55.285
>215, c1=7.590, c2=-55.274, g=-55.274
>216, c1=7.784, c2=-55.114, g=-55.114
>217, c1=8.005, c2=-55.026, g=-55.026
>218, c1=7.962, c2=-54.877, g=-54.877
>219, c1=7.470, c2=-54.81

>Saved: generated_plot_0291.png and wgan_results\model_0291.h5
>292, c1=9.031, c2=-46.236, g=-46.236
>293, c1=8.922, c2=-46.096, g=-46.096
>294, c1=8.924, c2=-46.021, g=-46.021
>295, c1=8.941, c2=-45.850, g=-45.850
>296, c1=9.058, c2=-45.718, g=-45.718
>297, c1=8.873, c2=-45.613, g=-45.613
>298, c1=9.160, c2=-45.422, g=-45.422
>299, c1=9.004, c2=-45.266, g=-45.266
>300, c1=8.925, c2=-45.196, g=-45.196
>301, c1=9.085, c2=-45.219, g=-45.219
>302, c1=9.175, c2=-45.047, g=-45.047
>303, c1=9.012, c2=-45.034, g=-45.034
>304, c1=8.863, c2=-44.978, g=-44.978
>305, c1=8.951, c2=-44.888, g=-44.888
>306, c1=9.153, c2=-44.689, g=-44.689
>307, c1=9.199, c2=-44.501, g=-44.501
>308, c1=9.277, c2=-44.501, g=-44.501
>309, c1=9.064, c2=-44.409, g=-44.409
>310, c1=9.205, c2=-44.266, g=-44.266
>311, c1=9.200, c2=-44.031, g=-44.031
>312, c1=9.127, c2=-43.872, g=-43.872
>313, c1=8.967, c2=-43.770, g=-43.770
>314, c1=9.006, c2=-43.659, g=-43.659
>315, c1=9.261, c2=-43.586, g=-43.586
>316, c1=9.216, c2=-43.49

>Saved: generated_plot_0388.png and wgan_results\model_0388.h5
>389, c1=9.162, c2=-35.990, g=-35.990
>390, c1=9.626, c2=-35.932, g=-35.932
>391, c1=9.320, c2=-35.861, g=-35.861
>392, c1=9.422, c2=-35.832, g=-35.832
>393, c1=9.567, c2=-35.731, g=-35.731
>394, c1=9.370, c2=-35.591, g=-35.591
>395, c1=9.507, c2=-35.453, g=-35.453
>396, c1=9.547, c2=-35.316, g=-35.316
>397, c1=9.482, c2=-35.186, g=-35.186
>398, c1=9.276, c2=-35.119, g=-35.119
>399, c1=9.460, c2=-35.044, g=-35.044
>400, c1=9.641, c2=-34.989, g=-34.989
>401, c1=9.569, c2=-34.868, g=-34.868
>402, c1=9.688, c2=-34.788, g=-34.788
>403, c1=9.462, c2=-34.717, g=-34.717
>404, c1=9.675, c2=-34.646, g=-34.646
>405, c1=9.296, c2=-34.583, g=-34.583
>406, c1=9.533, c2=-34.523, g=-34.523
>407, c1=9.415, c2=-34.473, g=-34.473
>408, c1=9.547, c2=-34.392, g=-34.392
>409, c1=9.427, c2=-34.303, g=-34.303
>410, c1=9.417, c2=-34.171, g=-34.171
>411, c1=9.493, c2=-34.142, g=-34.142
>412, c1=9.524, c2=-34.146, g=-34.146
>413, c1=9.597, c2=-34.07

...

>Saved: generated_plot_0582.png and wgan_results\model_0582.h5
>583, c1=9.266, c2=-26.394, g=-26.394
>584, c1=9.136, c2=-26.409, g=-26.409
>585, c1=9.110, c2=-26.411, g=-26.411
>586, c1=9.030, c2=-26.358, g=-26.358
>587, c1=9.181, c2=-26.286, g=-26.286
>588, c1=9.020, c2=-26.269, g=-26.269
>589, c1=9.095, c2=-26.270, g=-26.270
>590, c1=9.065, c2=-26.211, g=-26.211
>591, c1=9.120, c2=-26.170, g=-26.170
>592, c1=9.244, c2=-26.123, g=-26.123
>593, c1=9.124, c2=-26.115, g=-26.115
>594, c1=8.940, c2=-26.066, g=-26.066
>595, c1=9.129, c2=-26.003, g=-26.003
>596, c1=9.004, c2=-25.995, g=-25.995
>597, c1=9.032, c2=-25.967, g=-25.967
>598, c1=9.270, c2=-25.939, g=-25.939
>599, c1=8.949, c2=-25.935, g=-25.935
>600, c1=9.134, c2=-25.872, g=-25.872
>601, c1=8.944, c2=-25.868, g=-25.868
>602, c1=8.837, c2=-25.866, g=-25.866
>603, c1=9.102, c2=-25.839, g=-25.839
>604, c1=9.163, c2=-25.783, g=-25.783
>605, c1=8.966, c2=-25.775, g=-25.775
>606, c1=8.882, c2=-25.757, g=-25.757
>607, c1=9.044, c2=-25.69

>Saved: generated_plot_0679.png and wgan_results\model_0679.h5
>680, c1=8.805, c2=-24.317, g=-24.317
>681, c1=8.497, c2=-24.282, g=-24.282
>682, c1=8.603, c2=-24.285, g=-24.285
>683, c1=8.366, c2=-24.278, g=-24.278
>684, c1=8.716, c2=-24.231, g=-24.231
>685, c1=8.751, c2=-24.229, g=-24.229
>686, c1=8.577, c2=-24.199, g=-24.199
>687, c1=8.452, c2=-24.194, g=-24.194
>688, c1=8.616, c2=-24.187, g=-24.187
>689, c1=8.524, c2=-24.160, g=-24.160
>690, c1=8.582, c2=-24.107, g=-24.107
>691, c1=8.466, c2=-24.115, g=-24.115
>692, c1=8.627, c2=-24.120, g=-24.120
>693, c1=8.465, c2=-24.104, g=-24.104
>694, c1=8.424, c2=-24.113, g=-24.113
>695, c1=8.526, c2=-24.084, g=-24.084
>696, c1=8.745, c2=-24.066, g=-24.066
>697, c1=8.600, c2=-24.069, g=-24.069
>698, c1=8.494, c2=-24.077, g=-24.077
>699, c1=8.282, c2=-24.068, g=-24.068
>700, c1=8.507, c2=-24.047, g=-24.047
>701, c1=8.496, c2=-23.995, g=-23.995
>702, c1=8.497, c2=-23.973, g=-23.973
>703, c1=8.468, c2=-23.981, g=-23.981
>704, c1=8.378, c2=-23.97

>Saved: generated_plot_0776.png and wgan_results\model_0776.h5
>777, c1=8.363, c2=-22.931, g=-22.931
>778, c1=8.127, c2=-22.935, g=-22.935
>779, c1=7.951, c2=-22.889, g=-22.889
>780, c1=8.079, c2=-22.897, g=-22.897
>781, c1=7.934, c2=-22.893, g=-22.893
>782, c1=8.241, c2=-22.859, g=-22.859
>783, c1=8.074, c2=-22.822, g=-22.822
>784, c1=8.016, c2=-22.761, g=-22.761
>785, c1=8.227, c2=-22.731, g=-22.731
>786, c1=7.885, c2=-22.718, g=-22.718
>787, c1=8.055, c2=-22.691, g=-22.691
>788, c1=7.992, c2=-22.679, g=-22.679
>789, c1=8.006, c2=-22.688, g=-22.688
>790, c1=8.217, c2=-22.688, g=-22.688
>791, c1=8.439, c2=-22.693, g=-22.693
>792, c1=7.979, c2=-22.675, g=-22.675
>793, c1=8.000, c2=-22.682, g=-22.682
>794, c1=7.958, c2=-22.692, g=-22.692
>795, c1=8.124, c2=-22.656, g=-22.656
>796, c1=8.116, c2=-22.645, g=-22.645
>797, c1=8.015, c2=-22.626, g=-22.626
>798, c1=8.035, c2=-22.631, g=-22.631
>799, c1=8.072, c2=-22.616, g=-22.616
>800, c1=7.909, c2=-22.627, g=-22.627
>801, c1=8.102, c2=-22.62

>Saved: generated_plot_0873.png and wgan_results\model_0873.h5
>874, c1=7.757, c2=-21.765, g=-21.765
>875, c1=7.793, c2=-21.772, g=-21.772
>876, c1=7.733, c2=-21.754, g=-21.754
>877, c1=7.488, c2=-21.703, g=-21.703
>878, c1=7.717, c2=-21.684, g=-21.684
>879, c1=7.754, c2=-21.686, g=-21.686
>880, c1=7.621, c2=-21.656, g=-21.656
>881, c1=7.378, c2=-21.673, g=-21.673
>882, c1=7.674, c2=-21.650, g=-21.650
>883, c1=7.522, c2=-21.616, g=-21.616
>884, c1=7.508, c2=-21.619, g=-21.619
>885, c1=7.624, c2=-21.629, g=-21.629
>886, c1=7.731, c2=-21.644, g=-21.644
>887, c1=7.673, c2=-21.624, g=-21.624
>888, c1=7.718, c2=-21.583, g=-21.583
>889, c1=7.632, c2=-21.566, g=-21.566
>890, c1=7.741, c2=-21.572, g=-21.572
>891, c1=7.377, c2=-21.544, g=-21.544
>892, c1=7.436, c2=-21.541, g=-21.541
>893, c1=7.621, c2=-21.538, g=-21.538
>894, c1=7.716, c2=-21.529, g=-21.529
>895, c1=7.454, c2=-21.539, g=-21.539
>896, c1=7.688, c2=-21.503, g=-21.503
>897, c1=7.729, c2=-21.508, g=-21.508
>898, c1=7.558, c2=-21.48

>Saved: generated_plot_0970.png and wgan_results\model_0970.h5


#### References
1. [gans-in-action](https://github.com/GANs-in-Action/gans-in-action)
2. [Machine Learning Mastery](https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/)